In [7]:
import pandas as pd
import networkx as nx
import altair as alt
import nx_altair as nxa
import ast
from vega_datasets import data

In [8]:
df = pd.read_excel('CountryEconomics.xlsx')

# Test

In [9]:
gdp_bar_chart = alt.Chart(df).mark_bar().encode(
    x='Country',
    y='GDP'
)
gdp_bar_chart.show()

alt.Chart(...)

# Map

In [10]:
world_map = alt.topo_feature(data.world_110m.url, 'countries')

gdp_choropleth_map = alt.Chart(world_map).mark_geoshape().encode(
    color='GDP:Q',
    tooltip=['Country:N', 'GDP:Q']
).transform_lookup(
    lookup='id',
    from_=alt.LookupData(df, 'ID', ['Country', 'GDP'])
).project('mercator').properties(
    width=400,
    height=300,
    title='GDP by Country'
)
gdp_choropleth_map

alt.Chart(...)

# Nodes

In [11]:
# Parse Borders column to Python lists
df['Borders'] = df['Borders'].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) else ([] if pd.isna(x) else x)
)

# Create a set of valid 3-letter abbreviations in the dataset
valid_country_codes = set(df['Abbreviation_3'].dropna().tolist())

# Build the graph using 3-letter abbreviations as node IDs
border_graph = nx.Graph()
for idx, row in df.iterrows():
    country_code = row['Abbreviation_3']
    if pd.notna(country_code):  # Only add if we have a valid 3-letter code
        country_name = row['Country']
        # Add node with abbreviation as ID and country name as attribute
        border_graph.add_node(country_code, Country=country_name)
        # Add edges only for neighbors that exist in our dataset
        for neighbor_code in row['Borders']:
            if neighbor_code in valid_country_codes:
                border_graph.add_edge(country_code, neighbor_code)

# Visualize the network graph
node_positions = nx.spring_layout(border_graph, seed=42)
border_network_viz = nxa.draw_networkx(border_graph, node_positions, node_tooltip=['Country:N'])
border_network_viz = border_network_viz.properties(
    width=400,
    height=300,
    title='Country Border Network'
).interactive()
border_network_viz

alt.LayerChart(...)

# MCV

In [12]:
# Concatenate the GDP map and border network horizontally
combined_visualization = alt.hconcat(gdp_choropleth_map, border_network_viz)
combined_visualization

alt.HConcatChart(...)